# Hyperbolic tangent activation and dense network


In [1]:
import numpy as np
from math import sin
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
model = Sequential([
    Dense(20, input_dim=1, activation='tanh', use_bias=True),
    Dense(20, activation='tanh', use_bias=True),
    Dense(20, activation='tanh', use_bias=True),
    Dense(1, activation='tanh', use_bias=True)
])
model.compile(optimizer='adam', loss='mse')
model.summary()
    

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 20)                40        
_________________________________________________________________
dense_2 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_3 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 21        
Total params: 901
Trainable params: 901
Non-trainable params: 0
_________________________________________________________________


In [3]:
def gen_batch(batch_size):
    X = []
    Y = []
    for i in range(10):
        x = np.random.uniform(-np.pi/2, np.pi/2)
        y = sin(x)
        X.append(x)
        Y.append(y)
    return np.array([X, Y])

def gen(batch_size):
    while 1:
        yield gen_batch(batch_size)
        
model.fit_generator(generator=gen(20), steps_per_epoch=2000, epochs=20, validation_data=gen(5), validation_steps=10)

Epoch 1/20
2000/2000 [==============================] - 2s - loss: 0.0098 - val_loss: 0.0026
Epoch 2/20
2000/2000 [==============================] - 2s - loss: 0.0014 - val_loss: 1.2813e-04
Epoch 3/20
2000/2000 [==============================] - 2s - loss: 1.7105e-04 - val_loss: 2.5379e-04
Epoch 4/20
2000/2000 [==============================] - 2s - loss: 1.6541e-04 - val_loss: 1.1663e-04
Epoch 5/20
2000/2000 [==============================] - 2s - loss: 1.7426e-04 - val_loss: 1.3862e-04
Epoch 6/20
2000/2000 [==============================] - 2s - loss: 1.6659e-04 - val_loss: 9.4543e-05
Epoch 7/20
2000/2000 [==============================] - 2s - loss: 9.6002e-05 - val_loss: 4.1923e-04
Epoch 8/20
2000/2000 [==============================] - 2s - loss: 8.5434e-05 - val_loss: 4.0152e-05
Epoch 9/20
2000/2000 [==============================] - 2s - loss: 8.0384e-05 - val_loss: 3.6163e-05
Epoch 10/20
2000/2000 [==============================] - 2s - loss: 7.5336e-05 - val_loss: 5.5882e-05
E

In [6]:
x = [float(x-15)/10 for x in range(30)]
y = model.predict(np.array(x))
z = [sin(i) for i in x]

q = zip(x,[i[0] for i in y.tolist()],z)
for p in q:
    print(p)

(-1.5, -0.991264283657074, -0.9974949866040544)
(-1.4, -0.9815542101860046, -0.9854497299884601)
(-1.3, -0.9629166722297668, -0.963558185417193)
(-1.2, -0.9327337145805359, -0.9320390859672263)
(-1.1, -0.890956461429596, -0.8912073600614354)
(-1.0, -0.839418888092041, -0.8414709848078965)
(-0.9, -0.7801317572593689, -0.7833269096274834)
(-0.8, -0.7142901420593262, -0.7173560908995228)
(-0.7, -0.6421936750411987, -0.644217687237691)
(-0.6, -0.5636457800865173, -0.5646424733950354)
(-0.5, -0.47859710454940796, -0.479425538604203)
(-0.4, -0.3879101276397705, -0.3894183423086505)
(-0.3, -0.2937062680721283, -0.29552020666133955)
(-0.2, -0.19813098013401031, -0.19866933079506122)
(-0.1, -0.10085206478834152, -0.09983341664682815)
(0.0, -0.0005282257916405797, 0.0)
(0.1, 0.10003086924552917, 0.09983341664682815)
(0.2, 0.19779372215270996, 0.19866933079506122)
(0.3, 0.293769508600235, 0.29552020666133955)
(0.4, 0.38819190859794617, 0.3894183423086505)
(0.5, 0.47897103428840637, 0.479425538604

# Radial basis function

In [ ]:
# Idea taken from https://github.com/PetraVidnerova/rbf_keras

import numpy as np
from keras import backend as K
from keras.engine.topology import Layer
from keras.initializers import RandomUniform, Initializer, Orthogonal, Constant

class InitCentersRandom(Initializer):
  
    def __init__(self, X):
        self.X = X 

    def __call__(self, shape, dtype=None):
        print(shape)
        print(self.X.shape)
        assert shape[1] == self.X.shape[1]
        idx = np.random.randint(self.X.shape[0], size=shape[0])
        return self.X[idx,:]

        
class RBFLayer(Layer):
   
    def __init__(self, output_dim, initializer=RandomUniform(0.0, 1.0), betas=1.0, **kwargs):
        self.output_dim = output_dim
        super(RBFLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.centers = self.add_weight(shape=(self.output_dim, input_shape[1]),
                                       initializer=RandomUniform(0.0, 1.0),
                                       trainable=True)
        self.betas = self.add_weight(shape=(self.output_dim),
                                     initializer=Constant(value=1.0),
                                     trainable=True)
        super(RBFLayer, self).build(input_shape)  

    def call(self, x):
        C = K.expand_dims(self.centers)
        H = (C-x.T).T
        return K.exp(-self.betas * K.sum(H**2, axis=1))
   
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

    def get_config(self):
        config = {
            'output_dim': self.output_dim
        }
        base_config = super(RBFLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))
    
    
    
X = np.array(zip([float(x-5)/10 for x in range(10)],[float(x-5)/10 for x in range(10)]))

    
model = Sequential([
    RBFLayer(10, initializer=InitCentersRandom(X), betas=2.0,input_shape=(1,)),
    Dense(1)
])

model.fit_generator(generator=gen(20), steps_per_epoch=2000, epochs=20, validation_data=gen(5), validation_steps=10)